In [1]:
# 📦 Install
!pip install -q -U albumentations

# 📚 Imports
import numpy as np
import pandas as pd
import os
import cv2
import random
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import albumentations as A
from albumentations.core.composition import OneOf
from tensorflow.keras.utils import to_categorical

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.3/332.3 kB 7.3 MB/s eta 0:00:00:00:01


2025-05-15 10:53:48.157063: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747306428.640224      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747306428.758470      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# 💾 Dataset Paths
CSV_PATH = "/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv"
IMAGE_DIR_1 = "/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_1"
IMAGE_DIR_2 = "/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_2"

In [3]:
# 🧹 Load and preprocess data
df = pd.read_csv(CSV_PATH)


In [4]:
def get_image_path(image_id):
    path1 = os.path.join(IMAGE_DIR_1, f"{image_id}.jpg")
    path2 = os.path.join(IMAGE_DIR_2, f"{image_id}.jpg")
    return path1 if os.path.exists(path1) else path2


In [5]:
df['image_path'] = df['image_id'].apply(get_image_path)
df['label'] = LabelEncoder().fit_transform(df['dx'])

In [6]:
# ✅ Parameters
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 30
NUM_CLASSES = df['label'].nunique()
SEED = 42

In [7]:
# 🧪 Albumentations augmentations
from albumentations import (
    Resize, HorizontalFlip, RandomBrightnessContrast, HueSaturationValue,
    Affine, Normalize
)
from albumentations.augmentations.dropout.coarse_dropout import CoarseDropout

train_transform = A.Compose([
    Resize(IMG_SIZE, IMG_SIZE),
    HorizontalFlip(p=0.5),
    RandomBrightnessContrast(p=0.5),
    Affine(rotate=(-15, 15), scale=(0.95, 1.05), translate_percent=(0.0, 0.05), p=0.5),
    HueSaturationValue(p=0.4),
    CoarseDropout(max_holes=8, max_height=16, max_width=16, fill_value=0, p=0.3),
    Normalize()
])


val_transform = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.Normalize()
])


/tmp/ipykernel_35/286847742.py:14: UserWarning: Argument(s) 'max_holes, max_height, max_width, fill_value' are not valid for transform CoarseDropout
  CoarseDropout(max_holes=8, max_height=16, max_width=16, fill_value=0, p=0.3),


In [8]:
# 🧠 Image loader
def load_image(path, transform=None):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    if transform:
        image = transform(image=image)["image"]
    return image.astype(np.float32)

In [9]:
# 📦 Dataset generator
class SkinDataset(tf.keras.utils.Sequence):
    def __init__(self, image_paths, labels, batch_size, transform):
        self.image_paths = image_paths
        self.labels = labels
        self.batch_size = batch_size
        self.transform = transform
        self.indices = np.arange(len(self.image_paths))

    def __len__(self):
        return len(self.image_paths) // self.batch_size

    def __getitem__(self, idx):
        batch_paths = self.image_paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_labels = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]
        images = [load_image(p, self.transform) for p in batch_paths]
        return np.stack(images), to_categorical(batch_labels, num_classes=NUM_CLASSES)

    def on_epoch_end(self):
        np.random.shuffle(self.indices)
        self.image_paths = [self.image_paths[i] for i in self.indices]
        self.labels = [self.labels[i] for i in self.indices]

In [10]:
# 🔥 Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def loss_fn(y_true, y_pred):
        epsilon = 1e-9
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
        cross_entropy = -y_true * tf.math.log(y_pred)
        weight = alpha * tf.math.pow(1 - y_pred, gamma)
        return tf.reduce_mean(weight * cross_entropy)
    return loss_fn

In [11]:
# 🏗️ Build Model
def build_model():
    base = DenseNet201(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
    base.trainable = False

    x = base.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    output = Dense(NUM_CLASSES, activation='softmax')(x)

    model = Model(inputs=base.input, outputs=output)
    model.compile(optimizer='adam',
                  loss=focal_loss(gamma=2.0, alpha=0.25),
                  metrics=['accuracy'])
    return model

In [12]:
# 🔁 Stratified K-Fold Training
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

for fold, (train_idx, val_idx) in enumerate(skf.split(df, df['label'])):
    print(f"\n📂 Training Fold {fold + 1}")

    train_paths = df.iloc[train_idx]['image_path'].values
    val_paths = df.iloc[val_idx]['image_path'].values
    train_labels = df.iloc[train_idx]['label'].values
    val_labels = df.iloc[val_idx]['label'].values

    train_gen = SkinDataset(train_paths, train_labels, BATCH_SIZE, transform=train_transform)
    val_gen = SkinDataset(val_paths, val_labels, BATCH_SIZE, transform=val_transform)


📂 Training Fold 1

📂 Training Fold 2

📂 Training Fold 3

📂 Training Fold 4

📂 Training Fold 5


In [13]:
 model = build_model()

 callbacks = [
        EarlyStopping(patience=5, restore_best_weights=True),
        ReduceLROnPlateau(patience=3, factor=0.3, verbose=1)
    ]

 model.fit(train_gen,
              validation_data=val_gen,
              epochs=EPOCHS,
              callbacks=callbacks,
              verbose=1)

I0000 00:00:1747306476.261095      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1747306476.261863      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


74836368/74836368 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30


I0000 00:00:1747306515.520651     104 service.cc:148] XLA service 0x7a3d600e9dd0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747306515.522284     104 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1747306515.522306     104 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1747306520.129772     104 cuda_dnn.cc:529] Loaded cuDNN version 90300


  1/250 ━━━━━━━━━━━━━━━━━━━━ 3:54:19 56s/step - accuracy: 0.0625 - loss: 0.0784

I0000 00:00:1747306540.812274     104 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


250/250 ━━━━━━━━━━━━━━━━━━━━ 307s 1s/step - accuracy: 0.5690 - loss: 0.0358 - val_accuracy: 0.7293 - val_loss: 0.0158 - learning_rate: 0.0010
Epoch 2/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 109s 436ms/step - accuracy: 0.6884 - loss: 0.0205 - val_accuracy: 0.7455 - val_loss: 0.0130 - learning_rate: 0.0010
Epoch 3/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 114s 455ms/step - accuracy: 0.6898 - loss: 0.0183 - val_accuracy: 0.7692 - val_loss: 0.0122 - learning_rate: 0.0010
Epoch 4/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 107s 429ms/step - accuracy: 0.7033 - loss: 0.0177 - val_accuracy: 0.7737 - val_loss: 0.0121 - learning_rate: 0.0010
Epoch 5/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 109s 437ms/step - accuracy: 0.7019 - loss: 0.0177 - val_accuracy: 0.7661 - val_loss: 0.0123 - learning_rate: 0.0010
Epoch 6/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 107s 428ms/step - accuracy: 0.7096 - loss: 0.0163 - val_accuracy: 0.7727 - val_loss: 0.0120 - learning_rate: 0.0010
Epoch 7/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 105s 422ms/step - accuracy: 0.7043 - l

In [14]:
 model.save(f"densenet201_fold{fold+1}.h5")

In [15]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import cv2

In [16]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model(
        [model.inputs], 
        [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_output = predictions[:, pred_index]

    grads = tape.gradient(class_output, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [17]:
def overlay_heatmap(original_img, heatmap, alpha=0.4, colormap=cv2.COLORMAP_JET):
    heatmap = cv2.resize(heatmap, (original_img.shape[1], original_img.shape[0]))
    heatmap_colored = cv2.applyColorMap(np.uint8(255 * heatmap), colormap)
    overlayed_img = cv2.addWeighted(original_img, 1 - alpha, heatmap_colored, alpha, 0)
    return overlayed_img


In [18]:
def predict_and_visualize(image_path, model, label_map, last_conv_layer='conv5_block32_concat'):
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, (224, 224)) / 255.0
    img_input = np.expand_dims(img_resized, axis=0)

    # Prediction
    preds = model.predict(img_input)
    pred_class = np.argmax(preds[0])
    confidence = np.max(preds[0])

    # Grad-CAM
    heatmap = make_gradcam_heatmap(img_input, model, last_conv_layer)
    cam_image = overlay_heatmap(img_rgb, heatmap)

    # Display
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.title("Original Image")
    plt.imshow(img_rgb)
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.title(f"Grad-CAM: {label_map[pred_class]} ({confidence:.2f})")
    plt.imshow(cam_image)
    plt.axis('off')
    plt.show()

    return label_map[pred_class], confidence


In [21]:
from tensorflow.keras.models import load_model

# Example label map
label_map = {
    0: "Actinic keratoses",
    1: "Basal cell carcinoma",
    2: "Benign keratosis-like lesions",
    3: "Dermatofibroma",
    4: "Melanoma",
    5: "Melanocytic nevi",
    6: "Vascular lesions"
}

# Load model (update path if needed)
model = load_model("/kaggle/working/densenet201_fold5.h5", compile=False)

# Predict & visualize
predicted_label, confidence = predict_and_visualize(
    "/kaggle/input/ham10000-images-part-1/ISIC_0027419.jpg",  # update with your image file name
    model,
    label_map,
    conv_layer="top_activation"
  # DenseNet201 final conv layer
)

print(f"Prediction: {predicted_label}, Confidence: {confidence:.2f}")


TypeError: predict_and_visualize() got an unexpected keyword argument 'conv_layer'